## Install and Set up Kaggle

In [2]:
#install kaggle
! pip install -q kaggle

In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rohanbhatkande","key":"eec82f291e4ccd478f55538925982585"}'}

In [4]:
#create directory kaggle and copy json file to it, change permission of the file and download the dataset
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 26.8MB/s]
 99% 538M/543M [00:10<00:00, 54.4MB/s]
100% 543M/543M [00:10<00:00, 56.0MB/s]
 98% 266M/271M [00:04<00:00, 86.0MB/s]
100% 271M/271M [00:05<00:00, 56.6MB/s]


# unzip downloaded data

In [5]:
# create directory train and test and store the dataset in it
! mkdir train

! mkdir test

! unzip /content/train.zip -d train

! unzip /content/test1.zip -d test

Streaming output truncated to the last 5000 lines.
  inflating: test/test1/55.jpg       
  inflating: test/test1/550.jpg      
  inflating: test/test1/5500.jpg     
  inflating: test/test1/5501.jpg     
  inflating: test/test1/5502.jpg     
  inflating: test/test1/5503.jpg     
  inflating: test/test1/5504.jpg     
  inflating: test/test1/5505.jpg     
  inflating: test/test1/5506.jpg     
  inflating: test/test1/5507.jpg     
  inflating: test/test1/5508.jpg     
  inflating: test/test1/5509.jpg     
  inflating: test/test1/551.jpg      
  inflating: test/test1/5510.jpg     
  inflating: test/test1/5511.jpg     
  inflating: test/test1/5512.jpg     
  inflating: test/test1/5513.jpg     
  inflating: test/test1/5514.jpg     
  inflating: test/test1/5515.jpg     
  inflating: test/test1/5516.jpg     
  inflating: test/test1/5517.jpg     
  inflating: test/test1/5518.jpg     
  inflating: test/test1/5519.jpg     
  inflating: test/test1/552.jpg      
  inflating: test/test1/5520.jpg     

In [6]:
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# create directories
dataset_home = '/content/dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)
  
 
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = '/content/train/train/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = '/train'
	if random() < val_ratio:
		dst_dir = '/test'
	if file.startswith('cat'):
		dst = dataset_home + dst_dir + '/cats/'  + file
		copyfile(src, dst)
	elif file.startswith('dog'):
		dst = dataset_home + dst_dir + '/dogs/'  + file
		copyfile(src, dst)

# Data Creation(features and labels)
train and test dataset

In [7]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
#import pickle


file_list = []
class_list = []

DATADIR = "/content/dataset_dogs_vs_cats/train"
TESTDIR = "/content/dataset_dogs_vs_cats/test"

# All the categories you want to detect
CATEGORIES = [ "cats","dogs"]

# The size of the images
IMG_SIZE = 28

# Checking or all images in the data folder
for category in CATEGORIES :
	path = os.path.join(DATADIR, category)
	for img in os.listdir(path):
		img_array = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)

training_data = []
test_data = []
def create_training_data():
	for category in CATEGORIES :
		path = os.path.join(DATADIR, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
				training_data.append([new_array, class_num])
			except Exception as e:
				pass

def create_test_data():
	for category in CATEGORIES :
		path = os.path.join(TESTDIR, category)
		class_num = CATEGORIES.index(category)
		for img in os.listdir(path):
			try :
				img_array = cv2.imread(os.path.join(path, img),cv2.IMREAD_GRAYSCALE)
				new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
				test_data.append([new_array, class_num])
			except Exception as e:
				pass


create_training_data()
create_test_data()

random.shuffle(training_data)
random.shuffle(test_data)

X = [] #features
y = [] #labels

A = []
b = []

for features, label in training_data:
	X.append(features)
	y.append(label)
 
for features, label in test_data:
	A.append(features)
	b.append(label)
 
X_train = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE)
A_test= np.array(A).reshape(-1, IMG_SIZE, IMG_SIZE)

y_train = np.array(y)
b_test = np.array(b)


# Implementation of CNN

Convo Layer

In [8]:
import numpy as np

class Conv3x3:
  # A Convolution layer using 3x3 filters.

  def __init__(self, num_filters):
    self.num_filters = num_filters

    # filters is a 3d array with dimensions (num_filters, 3, 3)
    # We divide by 9 to reduce the variance of our initial values
    self.filters = np.random.randn(num_filters, 3, 3) / 9

  def iterate_regions(self, image):
    '''
    Generates all possible 3x3 image regions
    '''
    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the conv layer using the given input.
    '''
    self.last_input = input

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    return output

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the conv layer.
    - d_L_d_out is the loss gradient for this layer's outputs.
    '''
    d_L_d_filters = np.zeros(self.filters.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      for f in range(self.num_filters):
        d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

    # Update filters
    self.filters -= learn_rate * d_L_d_filters

    # We aren't returning anything here since we use Conv3x3 as the first layer in our CNN.
    # Otherwise, we'd need to return the loss gradient for this layer's inputs, just like every
    # other layer in our CNN.
    return None


Pooling Layer

In [9]:
class MaxPool2:
  # A Max Pooling layer using a pool size of 2.

  def iterate_regions(self, image):
    '''
    Generates non-overlapping 2x2 image regions to pool over.
    '''
    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the maxpool layer using the given input.

    '''
    self.last_input = input

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

  def backprop(self, d_L_d_out):
    '''
    Performs a backward pass of the maxpool layer.
    Returns the loss gradient for this layer's inputs.
    '''
    d_L_d_input = np.zeros(self.last_input.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      h, w, f = im_region.shape
      amax = np.amax(im_region, axis=(0, 1))

      for i2 in range(h):
        for j2 in range(w):
          for f2 in range(f):
            # If this pixel was the max value, copy the gradient to it.
            if im_region[i2, j2, f2] == amax[f2]:
              d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

    return d_L_d_input


Activation Function(Softmax)

In [10]:
class Softmax:
  # A standard fully-connected layer with softmax activation.

  def __init__(self, input_len, nodes):
    # We divide by input_len to reduce the variance of our initial values
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):
    '''
    Performs a forward pass of the softmax layer using the given input.

    '''
    self.last_input_shape = input.shape

    input = input.flatten()
    self.last_input = input

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    self.last_totals = totals

    exp = np.exp(totals)
    return exp / np.sum(exp, axis=0)

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the softmax layer.
    Returns the loss gradient for this layer's inputs.

    '''
    # We know only 1 element of d_L_d_out will be nonzero
    for i, gradient in enumerate(d_L_d_out):
      if gradient == 0:
        continue

      # e^totals
      t_exp = np.exp(self.last_totals)

      # Sum of all e^totals
      S = np.sum(t_exp)

      # Gradients of out[i] against totals
      d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
      d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

      # Gradients of totals against weights/biases/input
      d_t_d_w = self.last_input
      d_t_d_b = 1
      d_t_d_inputs = self.weights

      # Gradients of loss against totals
      d_L_d_t = gradient * d_out_d_t

      # Gradients of loss against weights/biases/input
      d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
      d_L_d_b = d_L_d_t * d_t_d_b
      d_L_d_inputs = d_t_d_inputs @ d_L_d_t

      # Update weights / biases
      self.weights -= learn_rate * d_L_d_w
      self.biases -= learn_rate * d_L_d_b

      return d_L_d_inputs.reshape(self.last_input_shape)


# Main Function

In [11]:
train_images= X_train[:6900]
train_labels= y_train[:6900]
test_images = A_test[:2400]
test_labels = b_test[:2400]

print(train_labels.shape)

print(train_images.shape)

conv = Conv3x3(8)                  
pool = MaxPool2()                  
softmax = Softmax(13 * 13 * 8, 10) 

def forward(image, label):
  
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy. 
  # np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

def train(im, label, lr=.005):
  
  #Completes a full training step on the given image and label.
  #Returns the cross-entropy loss and accuracy.
  
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

# Train the CNN for 10 epochs
for epoch in range(10):
  print('--- Epoch %d ---' % (epoch + 1))

  # Train!
  loss = 0
  num_correct = 0
  for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 100 == 99:
      print(
        '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i + 1, loss / 100, num_correct)
      )
      loss = 0
      num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct += acc

(6900,)
(6900, 28, 28)
--- Epoch 1 ---
[Step 100] Past 100 steps: Average Loss 2.000 | Accuracy: 52%
[Step 200] Past 100 steps: Average Loss 1.457 | Accuracy: 50%
[Step 300] Past 100 steps: Average Loss 1.146 | Accuracy: 59%
[Step 400] Past 100 steps: Average Loss 1.093 | Accuracy: 50%
[Step 500] Past 100 steps: Average Loss 0.965 | Accuracy: 62%
[Step 600] Past 100 steps: Average Loss 0.944 | Accuracy: 51%
[Step 700] Past 100 steps: Average Loss 0.886 | Accuracy: 54%
[Step 800] Past 100 steps: Average Loss 0.808 | Accuracy: 64%
[Step 900] Past 100 steps: Average Loss 0.836 | Accuracy: 53%
[Step 1000] Past 100 steps: Average Loss 0.803 | Accuracy: 54%
[Step 1100] Past 100 steps: Average Loss 0.813 | Accuracy: 45%
[Step 1200] Past 100 steps: Average Loss 0.766 | Accuracy: 55%
[Step 1300] Past 100 steps: Average Loss 0.770 | Accuracy: 52%
[Step 1400] Past 100 steps: Average Loss 0.762 | Accuracy: 56%
[Step 1500] Past 100 steps: Average Loss 0.730 | Accuracy: 56%
[Step 1600] Past 100 step

In [28]:
num_trains = len(train_images)
print("Train Loss:",loss/ num_trains)
print('Train Accuracy:', num_correct/ num_trains)

# Prediction and Testing

In [13]:
def forward_preds(image, label):
  
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy.
  # np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return np.argmax(out), loss, acc

In [24]:
# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
pred_labels= []
for im,label in zip(test_images, test_labels):
  predicted_lbl, l, acc = forward_preds(im,label)
  loss += l
  num_correct += acc
  pred_labels.append(predicted_lbl)


num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct/ num_tests)


--- Testing the CNN ---
Test Loss: 0.640870046535576
Test Accuracy: 0.6625


# Confusion matix for train and test error

In [15]:
from sklearn.metrics import confusion_matrix

print('The confusion matrix for test ')
confusion_matrix(test_labels, pred_labels)

The confusion matrix for test 


array([[684, 467],
       [343, 906]])

In [16]:
pred_labels = np.asarray(pred_labels)
pred_labels.shape

(2400,)

# Confusion matrix for Accuracy,Precision, Recall and F1 Score

In [17]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.67      0.59      0.63      1151
           1       0.66      0.73      0.69      1249

    accuracy                           0.66      2400
   macro avg       0.66      0.66      0.66      2400
weighted avg       0.66      0.66      0.66      2400

